In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from nltk.tokenize import word_tokenize
import jieba



import os
import sys
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.initializers import Constant

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#pre-train
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, '/Users/7ckung/Downloads/glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [3]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [4]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'cmn-eng/cmn.txt'

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    #target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for word in word_tokenize(input_text):
        if word not in input_words:
            input_words.add(word)
    for word in list(jieba.cut(target_text)):
        if word not in target_words:
            target_words.add(word)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/mf/4ngqf9b134d2m4zww7nmx6fc0000gn/T/jieba.cache
Loading model cost 0.777 seconds.
Prefix dict has been built succesfully.


In [7]:
input_words

{'lake',
 'Would',
 'backed',
 'drank',
 'sponge',
 'saw',
 'crazy',
 'skate',
 'Chinese',
 'knee',
 'absorbed',
 'talk',
 'Jealousy',
 'yard',
 'dropped',
 'radio',
 'fellow',
 'Divide',
 'throne',
 'trust',
 'wanting',
 'species',
 'sea',
 'Stop',
 'synchronize',
 'history',
 'forty',
 'An',
 'imitation',
 'Mexico',
 'marry',
 'cattle',
 'brother',
 '90',
 'team',
 'artificial',
 'drives',
 'shop',
 'Add',
 'mend',
 'undressing',
 'test',
 'elated',
 'hoped',
 'blamed',
 'exploded',
 'abandoned',
 'kick',
 'resembles',
 'dining',
 'stomachache',
 'password',
 'antiques',
 'discount',
 'afraid',
 'thirty-one',
 'guaranteed',
 'clever',
 'kitten',
 'tallest',
 'thieves',
 'putt',
 'planning',
 'third',
 '6',
 'watering',
 'Spanish',
 'directly',
 'pizza',
 'lights',
 'room',
 'disaster',
 'progress',
 'Day',
 'hurts',
 'depend',
 'homesick',
 'Canadians',
 'Electricity',
 'York',
 'dance',
 'flows',
 'clubs',
 'steps',
 'breaking',
 'useless',
 'elderly',
 'smart',
 'repay',
 'rates',


In [9]:
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(word_tokenize(txt)) for txt in input_texts])
max_decoder_seq_length = max([len(list(jieba.cut(txt))) for txt in target_texts])

In [10]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 3874
Number of unique output tokens: 6993
Max sequence length for inputs: 12
Max sequence length for outputs: 13


In [6]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(word_tokenize(input_text)):
        encoder_input_data[i, t, input_token_index[word]] = 1.
    for t, word in enumerate(jieba.cut(target_text)):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[word]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [8]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [12]:
input_token_index

{'!': 0,
 '$': 1,
 '%': 2,
 "''": 3,
 "'d": 4,
 "'ll": 5,
 "'m": 6,
 "'re": 7,
 "'s": 8,
 "'ve": 9,
 ',': 10,
 '.': 11,
 '1,000': 12,
 '10': 13,
 '10,000': 14,
 '100': 15,
 '105': 16,
 '10th': 17,
 '11': 18,
 '119': 19,
 '1636': 20,
 '17': 21,
 '18': 22,
 '1865': 23,
 '1945': 24,
 '1950': 25,
 '1970': 26,
 '1972': 27,
 '1988': 28,
 '1996': 29,
 '20': 30,
 '200': 31,
 '2003': 32,
 '2013': 33,
 '25th': 34,
 '2:30': 35,
 '3': 36,
 '3,000': 37,
 '30': 38,
 '3:30': 39,
 '4:00': 40,
 '5': 41,
 '58': 42,
 '6': 43,
 '65': 44,
 '6:00': 45,
 '6:30': 46,
 '7': 47,
 '7:00': 48,
 '7:30': 49,
 '8': 50,
 '800': 51,
 '839': 52,
 '8:30': 53,
 '9': 54,
 '90': 55,
 '9:00': 56,
 '?': 57,
 'A': 58,
 'ASAP': 59,
 'Above': 60,
 'Accidents': 61,
 'Add': 62,
 'Africa': 63,
 'After': 64,
 'All': 65,
 'Allow': 66,
 'Almost': 67,
 'Always': 68,
 'Am': 69,
 'America': 70,
 'American': 71,
 'An': 72,
 'Animals': 73,
 'Anniversaries': 74,
 'Answer': 75,
 'Any': 76,
 'Anybody': 77,
 'Anyone': 78,
 'Anything': 79,
 'A

In [9]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [12]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 437s 55ms/step - loss: 1.3211 - val_loss: 1.7009
Epoch 2/10
8000/8000 [==============================] - 482s 60ms/step - loss: 1.2391 - val_loss: 1.6733
Epoch 3/10
8000/8000 [==============================] - 473s 59ms/step - loss: 1.2033 - val_loss: 1.6492
Epoch 4/10
8000/8000 [==============================] - 450s 56ms/step - loss: 1.1691 - val_loss: 1.6396
Epoch 5/10
8000/8000 [==============================] - 432s 54ms/step - loss: 1.1382 - val_loss: 1.6467
Epoch 6/10
8000/8000 [==============================] - 431s 54ms/step - loss: 1.1051 - val_loss: 1.5956
Epoch 7/10
8000/8000 [==============================] - 479s 60ms/step - loss: 1.0698 - val_loss: 1.5868
Epoch 8/10
8000/8000 [==============================] - 467s 58ms/step - loss: 1.0386 - val_loss: 1.5709
Epoch 9/10
8000/8000 [==============================] - 469s 59ms/step - loss: 1.0072 - val_loss: 1.5593
Epoch 1

/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [13]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [14]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [17]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [18]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: 有一個的。。。。。。。。。。。。。。。。。
-
Input sentence: Hi.
Decoded sentence: 有一個的。。。。。。。。。。。。。。。。。
-
Input sentence: Run.
Decoded sentence: 是个的。。。。了。。。。。了。。。。了。。
-
Input sentence: Wait!
Decoded sentence: ！！吧。吧。吧。。。。。。。。。。。。。。
-
Input sentence: Hello!
Decoded sentence: ！！吧。吧。吧。。。。。。。。。。。。。。
-
Input sentence: I try.
Decoded sentence: 是个的。。。。。。。。。。。。。。。。。了
-
Input sentence: I won!
Decoded sentence: ！！！吧。吧。。。。。。。。。。。。。。。
-
Input sentence: Oh no!
Decoded sentence: ！！吧。吧。。。。。。。。。。。。。。。。
-
Input sentence: Cheers!
Decoded sentence: ！！吧。吧。吧。。。。。。。。。。。。。。
-
Input sentence: He ran.
Decoded sentence: 是个的。。。。。。。。。。。。。。。。了。
-
Input sentence: Hop in.
Decoded sentence: 是个的。。。。。。。。。。。。。。了。。。
-
Input sentence: I lost.
Decoded sentence: 是个的。。。了。。。。。。。。了。。。。了
-
Input sentence: I quit.
Decoded sentence: 是个的。。。。。。。。。。。。。。。了。。
-
Input sentence: I'm OK.
Decoded sentence: 是个的。。。。。。。。。。。。。。了。。。
-
Input sentence: Listen.
Decoded sentence: 是个好的。。。。。。。。。。。。。。。。。
-
Input sentence: No way!
D